In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
data = pd.read_csv('main dataset.csv')

In [5]:
#print(data.info())
columns_to_drop = ['Customer Email', 'Customer Password', 'Customer Street', 'Customer Zipcode', 
                   'Order City', 'Order Country', 'Order Customer Id', 'Order Region', 'Order Zipcode']
data = data.drop(columns=columns_to_drop)

In [7]:
from scipy.sparse import hstack

In [9]:
data = data.dropna()

def limit_categories(series, threshold=0.01):
    value_counts = series.value_counts(normalize=True)
    mask = series.isin(value_counts[value_counts >= threshold].index)
    return series.where(mask, other='Other')

if 'Customer City' in data.columns:
    data['Customer City'] = limit_categories(data['Customer City'])
if 'Product Name' in data.columns:
    data['Product Name'] = limit_categories(data['Product Name'])

if 'Customer Country' in data.columns:
    data['Customer Country'] = data['Customer Country'].astype(str)
if 'Customer State' in data.columns:
    data['Customer State'] = data['Customer State'].astype(str)

high_cardinality_columns = ['Customer Country', 'Customer State']
label_encoders = {}
for column in high_cardinality_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column].astype(str))
    label_encoders[column] = le

for column in data.select_dtypes(include=['object']).columns:
    data[column] = data[column].astype(str)

categorical_columns = data.select_dtypes(include=['object']).columns
one_hot_encoder = OneHotEncoder(sparse_output=True)
one_hot_encoded = one_hot_encoder.fit_transform(data[categorical_columns])

data = data.drop(columns=categorical_columns)
data_sparse = hstack([data.values, one_hot_encoded])

target = 'Days for shipment (scheduled)'
features = data.drop(columns=[target])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, data[target], test_size=0.2, random_state=42)

In [14]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Evaluate the Model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 0.4329954019167913


In [18]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plot_tree(model, feature_names=features.columns, filled=True, rounded=True,orientation='horizontal')
plt.gca().invert_yaxis()
plt.show()

TypeError: got an unexpected keyword argument 'orientation'

<Figure size 2000x1000 with 0 Axes>

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(20, 10))  
plot_tree(model, feature_names=features.columns, filled=True, rounded=True, ax=ax)

plt.gca().invert_yaxis()  
plt.show()

In [18]:
from sklearn.metrics import confusion_matrix , classification_report
cm= confusion_matrix(y_test, y_pred) 
print(cm)
print(classification_report(y_test, y_pred))

[[ 1988     0     0     0]
 [    0  5492    36    80]
 [    0    41  5281  1656]
 [    0   103  1787 19638]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1988
           1       0.97      0.98      0.98      5608
           2       0.74      0.76      0.75      6978
           4       0.92      0.91      0.92     21528

    accuracy                           0.90     36102
   macro avg       0.91      0.91      0.91     36102
weighted avg       0.90      0.90      0.90     36102



In [22]:
from sklearn.metrics import accuracy_score
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Predict on the test data
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print the accuracy on the training and test data
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Training Accuracy: 1.0000
Test Accuracy: 0.8974


In [24]:
import joblib
joblib.dump(model, 'deccisiontree.pkl')

['deccisiontree.pkl']

In [28]:
from sklearn.metrics import confusion_matrix , classification_report, accuracy_score
cm= confusion_matrix(y_test, y_pred) 
print(cm)
print(classification_report(y_test, y_pred))

y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

[[ 1988     0     0     0]
 [    0  5492    36    80]
 [    0    41  5281  1656]
 [    0   103  1787 19638]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1988
           1       0.97      0.98      0.98      5608
           2       0.74      0.76      0.75      6978
           4       0.92      0.91      0.92     21528

    accuracy                           0.90     36102
   macro avg       0.91      0.91      0.91     36102
weighted avg       0.90      0.90      0.90     36102

Training Accuracy: 1.0000
Test Accuracy: 0.8974
